In [1]:
%load_ext blackcellmagic
import sys
import uuid
sys.path.insert(0, "..")
def gen_uuid():
    return str(uuid.uuid4())[:8]

In [2]:
import algo
import coloredlogs
import graph
import importlib
import logging
import networkx as nx
import schedule as sch
import random
import topo
import yaml
importlib.reload(algo)
importlib.reload(graph)
importlib.reload(sch)
importlib.reload(topo)
coloredlogs.set_level(logging.INFO)

In [3]:
gen_args = {
    "graph_length": 5,
    "mi_cb": lambda: 10,
    "memory_cb": lambda: int(1e8),
    "unit_size_cb": lambda: random.randint(1000, 10000),
    "unit_rate_cb": lambda: random.randint(1, 100),
    "source_hosts": ["rasp1"],
    "sink_hosts": ["cloud1"],
}

In [4]:
sc = topo.Scenario.from_dict(yaml.load(open("../samples/a0.yaml", "r").read(), Loader=yaml.Loader))

In [5]:
gen_args_list = [
    {
        "graph_length": random.randint(3, 11),
        "mi_cb": lambda: 1,
        "memory_cb": lambda: int(1e8),
        "unit_size_cb": lambda: random.randint(10000, 100000),
        "unit_rate_cb": lambda: random.randint(1, 10),
        "source_hosts": ["rasp1"],
        "sink_hosts": ["cloud1"],
    }
    for _ in range(10)
]
graph_list = [graph.GraphGenerator(gen_uuid(), **gen_args).gen_random_chain_graph() for gen_args in gen_args_list]

In [6]:
sc.topo.clear_occupied()
min_cut_scheduler = sch.RandomScheduler()
min_cut_scheduler.logger.setLevel(logging.INFO)
min_cut_calculator = sch.LatencyCalculator(sc.topo)
min_cut_calculator.logger.setLevel(logging.INFO)
min_cut_result_list = []
for g in graph_list:
    s_cut, t_cut = algo.min_cut(g)
    print("graph {}: s {} t {}".format(g.uuid, len(s_cut), len(t_cut)))
    s_graph = g.sub_graph(s_cut, gen_uuid())
    t_graph = g.sub_graph(t_cut, gen_uuid())
    s_result = min_cut_scheduler.schedule(s_graph, sc.get_edge_domains()[0].topo)
    t_result = min_cut_scheduler.schedule(t_graph, sc.get_cloud_domains()[0].topo)
    if s_result.status == sch.SchedulingResultStatus.FAILED:
        print("s_graph {} failed: {}".format(g.uuid, s_result.reason))
        continue
    if t_result.status == sch.SchedulingResultStatus.FAILED:
        print("t_graph {} failed: {}".format(g.uuid, t_result.reason))
        continue
    result = sch.SchedulingResult.merge(s_result, t_result)
    min_cut_calculator.add_scheduled_graph(g, result)
min_cut_latency = min_cut_calculator.compute_latency()
print(min_cut_latency)
print(sum(min_cut_latency.values()))

graph 834f64c5: s 1 t 2
graph da369a50: s 2 t 2
graph bfc5ef50: s 1 t 2
graph e863e49e: s 1 t 6
graph 6888528a: s 2 t 5
graph 7ba368e4: s 3 t 5
graph bf7f03ac: s 8 t 1
graph 55a4fc5d: s 1 t 9
graph 86daf2e3: s 3 t 2
graph 495d7f93: s 1 t 3
{'834f64c5': 72.0, 'da369a50': 75.0, 'bfc5ef50': 75.0, 'e863e49e': 99.0, '6888528a': 78.0, '7ba368e4': 72.0, 'bf7f03ac': 75.0, '55a4fc5d': 78.0, '86daf2e3': 78.0, '495d7f93': 99.0}
801.0


In [7]:
sc.topo.clear_occupied()
random_scheduler = sch.RandomScheduler()
random_scheduler.logger.setLevel(logging.INFO)
random_calculator = sch.LatencyCalculator(sc.topo)
random_calculator.logger.setLevel(logging.INFO)
random_result_list = []
for g in graph_list:
    result = random_scheduler.schedule(g, sc.topo)
    if result.status == sch.SchedulingResultStatus.FAILED:
        print("graph {} failed: {}".format(g.uuid, result.reason))
    random_calculator.add_scheduled_graph(g, result)
random_latency = random_calculator.compute_latency()
print(random_latency)
print(sum(random_latency.values()))

{'834f64c5': 93.0, 'da369a50': 105.0, 'bfc5ef50': 127.0, 'e863e49e': 328.0, '6888528a': 504.0, '7ba368e4': 334.0, 'bf7f03ac': 328.0, '55a4fc5d': 592.0, '86daf2e3': 328.0, '495d7f93': 185.0}
2924.0


In [8]:
sc.topo.clear_occupied()
all_edge_scheduler = sch.RandomScheduler()
all_edge_scheduler.logger.setLevel(logging.INFO)
all_edge_calculator = sch.LatencyCalculator(sc.topo)
all_edge_calculator.logger.setLevel(logging.INFO)
all_edge_result_list = []
for g in graph_list:
    t_cut = set([v.uuid for v in g.get_sinks()])
    s_cut = set([v.uuid for v in g.get_vertexs()]) - t_cut
    s_graph = g.sub_graph(s_cut, gen_uuid())
    t_graph = g.sub_graph(t_cut, gen_uuid())
    s_result = all_edge_scheduler.schedule(s_graph, sc.get_edge_domains()[0].topo)
    t_result = all_edge_scheduler.schedule(t_graph, sc.get_cloud_domains()[0].topo)
    if s_result.status == sch.SchedulingResultStatus.FAILED:
        print("s_graph {} failed: {}".format(g.uuid, s_result.reason))
        continue
    if t_result.status == sch.SchedulingResultStatus.FAILED:
        print("t_graph {} failed: {}".format(g.uuid, t_result.reason))
        continue
    result = sch.SchedulingResult.merge(s_result, t_result)
    all_edge_calculator.add_scheduled_graph(g, result)
all_edge_latency = all_edge_calculator.compute_latency()
print(all_edge_latency)
print(sum(all_edge_latency.values()))

{'834f64c5': 90.0, 'da369a50': 87.0, 'bfc5ef50': 165.0, 'e863e49e': 90.0, '6888528a': 90.0, '7ba368e4': 93.0, 'bf7f03ac': 105.0, '55a4fc5d': 96.0, '86daf2e3': 97.0, '495d7f93': 88.0}
1001.0
